
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Lab: Navigating the Metastore
In this demo, we'll explore the structure and functionality of a metastore, delving into its various components like catalogs, schemas, and tables. We'll employ SQL commands such as SHOW and DESCRIBE to inspect and analyze these elements, enhancing our understanding of the metastore's configuration and the properties of different data objects. Additionally, we'll examine the roles of system catalogs and information_schema in metadata management, and highlight the importance of data lineage in data governance. This hands-on demonstration will equip participants with the knowledge to effectively navigate and utilize metastores in a cloud environment.

### Learning Objectives
By the end of this demo, you will be able to:
1. Discuss the structure and function of a metastore, including its different components such as catalogs, schemas, and tables.
2. Apply SQL commands like `SHOW` and `DESCRIBE` to inspect and explore different elements within the metastore, such as catalogs, schemas, tables, user-defined functions, and privileges.
3. Analyze and interpret the configuration of the metastore and the properties of various data objects.
4. Evaluate the roles of the system catalog and the information_schema in managing and accessing metadata.
5. Identify and explain the importance of data lineage as part of data governance.


## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your specific catalog and the schema to the schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG <your catalog>;
USE SCHEMA <your catalog>.<schema>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-2

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Created the silver table and vw_gold view in your catalog labuser9907309_1744210882 with the example schema.
Set the default catalog to labuser9907309_1744210882.
Set the default schema to example.


Your Unity Catalog name:,
Your Schema Name:,


## A. Analyze Data Objects in Classroom Setup
Let us analyze the current data objects and their components during the classroom setup.

### A1. Analyze the Current Catalog

1. Run code to view your current default catalog. Confirm that the catalog name displayed above is your current catalog.


In [0]:
SELECT current_catalog();

current_catalog()
labuser9907309_1744210882


### A2. Analyze the Current Schema

1. Run code to view your current default schema. Confirm that your current schema is **example**.


In [0]:
SELECT current_schema();

current_schema()
example


### A3. Analyze the List of Available Tables and Views in the Custom Schema

1. Let us analyze your **example** schema to display a list of tables and views. Confirm that the schema contains the **silver** table and the **vw_gold** view.


In [0]:
SHOW TABLES IN example;

database,tableName,isTemporary
example,silver,false
example,vw_gold,false


2. Display the available views in your current schema. Confirm the schema contains the view **vw_gold**.

In [0]:
SHOW VIEWS IN example

namespace,viewName,isTemporary,isMaterialized
example,vw_gold,false,false


**NOTE:** `SHOW TABLES` will display both tables and views, and `SHOW VIEWS` will only show views. From the above observation, there are the following tables and views in the custom schema:
1. Table\(s\): **silver**
2. View\(s\): **vw_gold**


## B. Exploring the Metastore

In this section, let's explore our metastore and its data objects.


### Using SQL: Inspect Elements with SQL `SHOW` Command
Let's explore objects using the SQL commands. Though we embed them in a notebook here, you can easily port them over for execution in the DBSQL environment as well.

We use the SQL `SHOW` command to inspect elements at various levels in the hierarchy.

For syntax references, check out the [SQL language reference - DDL statements](https://docs.databricks.com/en/sql/language-manual/index.html#ddl-statements)

#### B1. Inspect Catalogs
1. Let's start by displaying all available catalogs in the metastore with the `SHOW` statement. Confirm a variety of catalogs exist.

In [0]:
SHOW CATALOGS

catalog
dbacademy
dbacademy_movies
dbacademy_retail
hive_metastore
labuser9907309_1744210882
main
samples
system


Do any of these entries surprise you? You should definitely see a catalog beginning with your user name as the prefix, which is the one we created earlier. But there may be more, depending on the activity in your metastore, and how your workspace is configured. In addition to catalogs others have created, you will also see some special catalogs:
* **hive_metastore**. This is not actually a catalog. Rather, it's Unity Catalog's way of making the workspace local Hive metastore seamlessly accessible through the three-level namespace.
* **main**: this catalog is created by default with each new metastore, though you can remove it from your metastore if desired (it isn't used by the system)
* **samples**: this references a cloud container containing sample datasets hosted by Databricks.
* **system**: this catalog provides an interface to the system tables - a collection of tables that return information about objects across all catalogs in the metastore.


#### B2. Inspect Schemas
1. Now let's take a look at the schemas contained in your specific catalog (your default catalog). Remember that we have a default catalog selected so we needn't specify it in our query. Confirm the schemas **default**, **dmguc**, **example** and **information_schema** exist.

In [0]:
SHOW SCHEMAS

databaseName
default
example
information_schema


The **example** schema, of course, is the one we created earlier but there are a couple additional entries you maybe weren't expecting:
* **default**: this schema is created by default with each new catalog.
* **information_schema**: this schema is also created by default with each new catalog and provides a set of views describing the objects in the catalog.

2. As a sidenote, if we want to inspect schemas in a catalog that isn't the default, we specify it as follows `SHOW SCHEMAS IN catalog-name`. Run code to view available schemas in the **samples** catalog. Confirm multiple schemas exist.

In [0]:
SHOW SCHEMAS IN samples;

databaseName
accuweather
bakehouse
information_schema
nyctaxi
tpch


#### B3. Inspect Tables
1. Now let's take a look at the tables contained our  **example** schema within our course catalog. Again, we don't need to specify schema or catalog since we're referencing the defaults. Confirm the **silver** table and **vw_gold** view exist.

In [0]:
SHOW TABLES

database,tableName,isTemporary
example,silver,false
example,vw_gold,false


2. If you want to inspect elsewhere, you can explicitly override the default catalog and schema as follows: `SHOW TABLES IN catalog-name.schema-name`. 

   View the available tables in the **samples** catalog within the **tpch** schema. Confirm that a variety of tables are available.

In [0]:
SHOW TABLES IN samples.tpch

database,tableName,isTemporary
tpch,customer,false
tpch,lineitem,false
tpch,nation,false
tpch,orders,false
tpch,part,false
tpch,partsupp,false
tpch,region,false
tpch,supplier,false


#### B4. Inspect User-Defined Functions
1. There's a command available for exploring all the different object types. For example, display the available user-defined functions in your default schema (**examples**). Confirm that the `dbacademy_mask` function is available.

[SHOW FUNCTIONS](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-aux-show-functions.html)


In [0]:
SHOW USER FUNCTIONS


function
getargument
labuser9907309_1744210882.example.dbacademy_mask


#### B5. Inspect Privileges Granted on Data Objects
We can also use `SHOW` to see privileges granted on data objects.

For syntax references, check out the [SQL language reference - Security statements](https://docs.databricks.com/en/sql/language-manual/index.html#security-statements) documentation.

1. Display all privileges (inherited, denied, and granted) on your **silver** table in the **examples** schema (default schema). Confirm that *ALL PRIVILEGES* are available to your user account.



**HINT:** `SHOW GRANTS ON`

In [0]:
SHOW GRANTS ON TABLE silver

Principal,ActionType,ObjectType,ObjectKey
labuser9907309_1744210882@vocareum.com,ALL PRIVILEGES,CATALOG,labuser9907309_1744210882


Since there are no grants on this table yet, no results are returned. That means that only you, the data owner, can access this table. We'll get to granting privileges shortly.

## C. Analyze Additional Information with SQL `DESCRIBE` Command

We also have `DESCRIBE` at our disposal, to provide additional information about a specific object.

For syntax references, check out the [SQL language reference](https://docs.databricks.com/en/sql/language-manual/index.html#sql-language-reference) documentation.

### C1. Analyze Tables
1. Let us analyze the information about a few tables. 

    Use the `DESCRIBE TABLE EXTENDED` statement on your **silver** table to display detailed information about the specified columns, including the column statistics collected by the command, and additional metadata information (such as schema qualifier, owner, and access time).

In [0]:
DESCRIBE TABLE EXTENDED silver

col_name,data_type,comment
device_id,int,null
mrn,string,null
name,string,null
time,timestamp,null
heartrate,double,null
,,
# Delta Statistics Columns,,
Column Names,"device_id, name, time, mrn, heartrate",
Column Selection Method,first-32,
,,


2. Use the same statement as above to view information about your **vw_gold** view. In the results, scroll down to the *View Text* value in the **col_name** column. Notice that you can view the SQL text for the view.


In [0]:
DESCRIBE TABLE EXTENDED vw_gold

col_name,data_type,comment
mrn,string,null
name,string,null
avg_heartrate,double,null
date,timestamp,null
,,
# Detailed Table Information,,
Catalog,labuser9907309_1744210882,
Database,example,
Table,vw_gold,
Owner,labuser9907309_1744210882@vocareum.com,


### C2. Analyze User-Defined Functions
1. Let us analyze the information about the **dbacademy_mask** user-defined function in the **example** schema. Use the `DESCRIBE FUNCTION EXTENDED` statement to view detailed information about the function.

In [0]:
DESCRIBE FUNCTION EXTENDED dbacademy_mask

function_desc
Function: labuser9907309_1744210882.example.dbacademy_mask
Type: SCALAR
Input: x STRING
Returns: STRING
Deterministic: true
Data Access: CONTAINS SQL
Configs: spark.databricks.sql.expression.aiFunctions.repartition=0
spark.databricks.sql.functions.aiForecast.enabled=false
spark.databricks.sql.functions.aiFunctions.adaptiveThreadPool.clusterSizeBasedGlobalParallelism.scaleFactor=512.0
spark.databricks.sql.functions.aiFunctions.adaptiveThreadPool.debugLogEnabled=true


## D. Analyze Other Data Objects
We can also analyze other data objects in the metastore.

### D1. System Catalog
The *system* catalog provides an interface to the system tables; that is a collection of views whose purpose is to provide a SQL-based, self-describing API to the metadata related to objects across all catalogs in the metastore. This exposes a host of information useful for administration and housekeeping and there are a lot of applications for this.



1. Run the following cell to view tables in the **system** catalog's **information_schema** schema. Notice that a variety of system tables are available.

   **NOTE:** System tables are a Databricks-hosted analytical store of your account’s operational data found in the system catalog. These tables can be used for historical observability across your account.

   For more information, check out the [Monitor usage with system tables](https://docs.databricks.com/en/admin/system-tables/index.html#monitor-usage-with-system-tables) documentation.


In [0]:
SHOW TABLES in system.information_schema;

database,tableName,isTemporary
information_schema,catalog_privileges,false
information_schema,catalog_provider_share_usage,false
information_schema,catalog_tags,false
information_schema,catalogs,false
information_schema,check_constraints,false
information_schema,column_masks,false
information_schema,column_tags,false
information_schema,columns,false
information_schema,connection_privileges,false
information_schema,connections,false


2. Let's consider the following query, which shows all tables that have been modified in the last *24 hours* using the **system** catalog.

    **NOTE:** In addition to demonstrating how to leverage this information, the query also demonstrates a Unity Catalog three-level namespace reference.

In [0]:
SELECT 
    table_name, 
    table_owner, 
    created_by, 
    last_altered, 
    last_altered_by, 
    table_catalog
FROM system.information_schema.tables
WHERE  datediff(now(), last_altered) < 1;

table_name,table_owner,created_by,last_altered,last_altered_by,table_catalog
catalog_privileges,System user,System user,2025-04-09T15:02:03.081Z,System user,labuser9907309_1744210882
views,System user,System user,2025-04-09T15:02:03.522Z,System user,labuser9907309_1744210882
meta,metastore_admins,9556a37f-7dc0-4b5f-849c-babbde9b34af,2025-04-09T15:02:01.877Z,9556a37f-7dc0-4b5f-849c-babbde9b34af,dbacademy
routine_columns,System user,System user,2025-04-09T15:02:03.276Z,System user,labuser9907309_1744210882
customers,metastore_admins,9556a37f-7dc0-4b5f-849c-babbde9b34af,2025-04-09T10:37:27.937Z,System user,dbacademy_retail
table_privileges,System user,System user,2025-04-09T15:02:03.477Z,System user,labuser9907309_1744210882
routine_privileges,System user,System user,2025-04-09T15:02:03.324Z,System user,labuser9907309_1744210882
table_constraints,System user,System user,2025-04-09T15:02:03.391Z,System user,labuser9907309_1744210882
table_tags,System user,System user,2025-04-09T15:02:03.667Z,System user,labuser9907309_1744210882
catalog_tags,System user,System user,2025-04-09T15:02:03.628Z,System user,labuser9907309_1744210882


### D2. Information Schema

The *information_schema* is automatically created with each catalog and contains a collection of views whose purpose is to provide a SQL-based, self-describing API to the metadata related to the elements contained within the catalog.

The relations found in this schema are documented <a href="https://docs.databricks.com/sql/language-manual/sql-ref-information-schema.html" target="_blank">here</a>. 

1. As a basic example, let's see all of your available tables in your default catalog. Note that since we only specify two levels here, we're referencing the default catalog selected earlier.

In [0]:
SELECT * FROM information_schema.tables

table_catalog,table_schema,table_name,table_type,is_insertable_into,commit_action,table_owner,comment,created,created_by,last_altered,last_altered_by,data_source_format,storage_sub_directory,storage_path
labuser9907309_1744210882,information_schema,catalog_privileges,VIEW,NO,PRESERVE,System user,Lists principals that have privileges on the catalogs.,2025-04-09T15:02:03.081Z,System user,2025-04-09T15:02:03.081Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,views,VIEW,NO,PRESERVE,System user,Describes view specific information about the views in the catalog.,2025-04-09T15:02:03.522Z,System user,2025-04-09T15:02:03.522Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,routine_columns,VIEW,NO,PRESERVE,System user,Describes result columns of table valued functions.,2025-04-09T15:02:03.276Z,System user,2025-04-09T15:02:03.276Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,table_privileges,VIEW,NO,PRESERVE,System user,Lists principals that have privileges on the tables and views in the catalog.,2025-04-09T15:02:03.477Z,System user,2025-04-09T15:02:03.477Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,routine_privileges,VIEW,NO,PRESERVE,System user,Lists principals that have privileges on the routines in the catalog.,2025-04-09T15:02:03.324Z,System user,2025-04-09T15:02:03.324Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,table_constraints,VIEW,NO,PRESERVE,System user,Describes metadata for all primary and foreign key constraints within the catalog.,2025-04-09T15:02:03.391Z,System user,2025-04-09T15:02:03.391Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,table_tags,VIEW,NO,PRESERVE,System user,null,2025-04-09T15:02:03.667Z,System user,2025-04-09T15:02:03.667Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,catalog_tags,VIEW,NO,PRESERVE,System user,null,2025-04-09T15:02:03.628Z,System user,2025-04-09T15:02:03.628Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,schema_privileges,VIEW,NO,PRESERVE,System user,Lists principals that have privileges on the schemas in the catalog.,2025-04-09T15:02:03.346Z,System user,2025-04-09T15:02:03.346Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null
labuser9907309_1744210882,information_schema,volume_tags,VIEW,NO,PRESERVE,System user,null,2025-04-09T15:02:03.708Z,System user,2025-04-09T15:02:03.708Z,System user,UNKNOWN_DATA_SOURCE_FORMAT,null,null


## E. Catalog Explorer


### E1. Open the Catalog Explorer
1. Right-click on **Catalog** in the left sidebar to explore the metastore using the Catalog Explorer user interface.
   
   1. Observe the catalogs listed in the **Catalog** pane and select *Open Link in New Tab*.

   2. The items in this list resemble those from the `SHOW CATALOGS` SQL statement we executed earlier.

   3. Expand your unique catalog name, then expand **example**. This displays a list of tables, views, and functions.
   
   4. Expand **tables**, then select **vw_gold** to see detailed information regarding the view. 
   
   5. From here, you can view the schema, sample data, details, and permissions (which we'll get to shortly).


### E2. Lineage

Data lineage is a key pillar of any data governance solution.

1. Select the **Lineage** tab.

2. In the **Lineage** tab, you can identify elements related to the selected object.

3. Select the **See lineage graph** button in the upper left corner.

4. The lineage graph provides a visualization of the lineage relationships.


## Conclusion
In this demo, we explored the structure and functionality of a metastore through practical exercises, enhancing our understanding of data organization and metadata management. We learned how to navigate and inspect various components such as catalogs, schemas, tables, and user-defined functions using SQL commands like SHOW and DESCRIBE. Additionally, we delved into the roles of the system catalog and information_schema, gaining insights into their importance in metadata access and management. The demo also highlighted the significance of data lineage for robust data governance, enabling us to trace data origins and impacts effectively. Overall, this hands-on approach has equipped us with essential skills to manage and analyze metadata within a metastore efficiently.


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>